# FlowDenoising User Manual

FlowDenoising is a low-pass filtering tool for volumetric data. A 1D Gaussian filter is applied to each dimension Z, Y and X (in this order), resulting in a 3D filtering. Unlike classical Gaussian filtering, the 2D slices (in each dimension) are aligned to minimize the smoothing of the structures recognized by the Optical Flow (OF) estimator.

## 1. Installing the required packages

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
# Delete temporal files
!rm -f *.mrc *.tif

## 2. Denoising using the default parameters

The default parameters are:

In [ ]:
!python ../flowdenoising.py -h

### Download a (small) volume
This example uses a 32x256x256 (Z, Y, X) crop of the [EMPIAR-10311](https://www.ebi.ac.uk/empiar/EMPIAR-10310/) volume.

In [ ]:
%%bash
OUTPUT_FILENAME="small_vol.mrc"
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1jYL6FEMeWGXO0KYlCb9udrICc2qaZLHB"
    wget --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O $OUTPUT_FILENAME
fi

In [ ]:
!ls -l *.mrc

### Filter (using [MRC](https://en.wikipedia.org/wiki/MRC_(file_format)) files)

In [ ]:
%%time
!python ../flowdenoising.py -i small_vol.mrc -o denoised_vol.mrc

In [ ]:
!ls -l *.mrc

### Filter (using multi-page [TIFF](https://en.wikipedia.org/wiki/TIFF) files)

In [ ]:
%%time
!python ../flowdenoising.py -i small_vol.mrc -o denoised_vol.tif

In [ ]:
!ls -l *.tif

### Filtering without optical flow (OF) compensation

In [ ]:
%%time
!python ../flowdenoising.py -i small_vol.mrc -o no_OF_denoised_vol.tif --no_OF 

In [ ]:
!ls -l *.tif

### Compare

In [ ]:
import skimage.io
import mrcfile
vol_MRC = mrcfile.open("small_vol.mrc")
vol = vol_MRC.data
denoised_vol = skimage.io.imread("denoised_vol.tif", plugin="tifffile")
no_OF_denoised_stack = skimage.io.imread("no_OF_denoised_vol.tif", plugin="tifffile")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
from IPython.display import display, Math
display(Math("\mathtt{Original~slice}:"))
imgplot = plt.imshow(vol[15], cmap="gray")

In [ ]:
display(Math("\mathtt{Denoised~slice~using~OF~}(\sigma=2):"))
imgplot = plt.imshow(denoised_vol[15], cmap="gray")

In [ ]:
display(Math("\mathtt{Denoised~slice~using~standard~Gaussian~filtering~}(\sigma=2):"))
imgplot = plt.imshow(no_OF_denoised_vol[15], cmap="gray")

### Download a ("big") volume

In [ ]:
%%bash
OUTPUT_FILENAME="big_vol.mrc"
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1lSFb4VZ5iqR3kLdPv1dDbdOe9dKPZ--_"
    wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O $OUTPUT_FILENAME && rm -rf /tmp/cookies.txt
fi

In [ ]:
vol_MRC = mrcfile.open("big_vol.mrc")
vol = vol_MRC.data
print("(Z_size, Y_size, X_size) =", vol.shape)

### Filter

In [ ]:
%%time
!python ../flowdenoising.py -i big_vol.mrc -o denoised_big_vol.mrc

### Filter without OF compensation

In [ ]:
%%time
!python ../flowdenoising.py -i big_vol.mrc -o no_OF_denoised_bit_vol.mrc --no_OF 

### Effect

In [ ]:
denoised_vol_MRC = mrcfile.open("denoised_big_vol.mrc")
denoised_vol = denoised_vol_MRC.data
no_OF_denoised_vol_MRC = mrcfile.open("no_OF_denoised_big_vol.mrc")
no_OF_denoised_vol = no_OF_denoised_vol_MRC.data

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(12, 12))
display(Math("\mathtt{Original~slice}:"))
imgplot = plt.imshow(vol[vol.shape[0]//2], cmap="gray")

In [ ]:
display(Math("\mathtt{Denoised~slice~using~OF}~(\sigma=2):"))
figure(figsize=(12, 12))
imgplot = plt.imshow(denoised_vol[vol.shape[0]//2], cmap="gray")

In [ ]:
display(Math("\mathtt{Denoised~slice~using~standard~Gaussian~filtering}~(\sigma=2):"))
figure(figsize=(12, 12))
imgplot = plt.imshow(no_OF_denoised_vol[vol.shape[0]//2], cmap="gray")

## 3. Controlling the length of the filters ($\sigma$)

### $\sigma_\text{Z}=1.0, \sigma_\text{Y}=2.0, \sigma_{X}=3.0$

In [ ]:
%%time
!python ../flowdenoising.py -i small_vol.mrc -o denoised_vol.mrc -s 